In [1]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
import os

load_dotenv(override=True) # take environment variables from .env.

# Variables not used here do not need to be updated in your .env file
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
key_credential = os.environ["AZURE_SEARCH_ADMIN_KEY"] if len(os.environ["AZURE_SEARCH_ADMIN_KEY"]) > 0 else None
index_name = os.environ["AZURE_SEARCH_INDEX"]
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.environ["AZURE_OPENAI_API_KEY"] if len(os.environ["AZURE_OPENAI_API_KEY"]) > 0 else None
azure_openai_embedding_deployment = os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"]
azure_openai_api_version = os.environ["AZURE_OPENAI_API_VERSION"]

credential = key_credential or DefaultAzureCredential()

In [2]:
from langchain_openai import AzureOpenAIEmbeddings
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from langchain.vectorstores.azuresearch import AzureSearch

openai_credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(openai_credential, "https://cognitiveservices.azure.com/.default")

# Use API key if provided, otherwise use RBAC authentication
embeddings = AzureOpenAIEmbeddings(
    azure_deployment=azure_openai_embedding_deployment,
    openai_api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key,
    azure_ad_token_provider=token_provider if not azure_openai_key else None
)   

vector_store = AzureSearch(
    azure_search_endpoint=endpoint,
    azure_search_key=key_credential,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
    semantic_configuration_name="contoso-semantic-config"
) 

In [3]:
# If you have langsmith access, run the following to trace and debuy your agent

from langsmith import Client
client = Client()

os.environ['LANGCHAIN_PROJECT'] = "contoso-agent"

In [10]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import tool

# Define the input schema
class QueryInput(BaseModel):
    question: str = Field(description="Question inputted by the user to retrieve the top documents from the Crypto vector store or knowledge base")
    k: int = Field(default=10, description="The number of top documents to retrieve")

    
@tool(args_schema=QueryInput)  
def retrieve(question:str, k:int=10) -> str:  
    """Retrieve the top k documents from the Crypto vector store based on the user question"""  
      
    docs = vector_store.hybrid_search(question, k=k)  
    context = "\n\n".join([doc.page_content for doc in docs])  
    return context  

In [5]:
@tool  
def retrieve(question:str, k:int=15) -> str:  
    """
    Retrieve the top k documents from the Crypto vector store based on the user question.
    
    Parameters:  
    question (str): The user question to search the documents for.  
    k (int, optional): The number of top documents to retrieve. Default is 15.  
  
    Returns:  
    str: A string representing the context relevant to the user question, made by concatenating the page_content of the top 'k' documents. 
    """        
    docs = vector_store.hybrid_search(question, k=k)  
    context = "\n\n".join([doc.page_content for doc in docs])  
    return context  

print(retrieve("What French coffee options and bakery items do you have?"))

c:\Users\shchitt\AppData\Local\anaconda3\envs\langgraph\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 
        Item Name: Pain au Chocolat
        Pain au Chocolat details:
            Price: 3.5$
            Category: Bakery
            Description: Classic French pastry filled with chocolate.
    

 
        Item Name: Cinnamon Roll
        Cinnamon Roll details:
            Price: 4.0$
            Category: Bakery
            Description: Sweet roll covered in delicious cinnamon and sugar.
    

 
        Item Name: Cafe Breve
        Cafe Breve details:
            Price: 4.0$
            Category: Coffees
            Description: A coffee drink made with espresso and half-and-half instead of milk.
    

 
        Item Name: Scones
        Scones details:
            Price: 3.0$
            Category: Bakery
            Description: Buttery and crumbly traditional English baked good.
    

 
        Item Name: Cafe au Lait
        Cafe au Lait details:
            Price: 3.5$
            Category: Coffees
            Description: A coffee drink made with strong hot coffee and scald

In [11]:
from typing import List, Dict, Optional
from pydantic.v1 import BaseModel, Field, root_validator, ValidationError
from langchain.agents import tool

class OrderedItem(BaseModel):
    """
    Information about each ordered item such as item name, quantity, price and additional notes provided by the customer.
    """
    item: str = Field(description="Name of the item ordered")
    quantity: int = Field(description="Quantity of that ordered item")
    price: float = Field(description="Price of that ordered item")
    item_notes: Optional[str] = Field(description="Any additional notes about that item provided by the customer")
    
class OrderInfo(BaseModel):
    """
    Input is a transcript of a conversation between a customer and an AI assistant discussing an order made at a restaurant.
    Extract order details such item name, quantity, price, and any additional notes for each item.
    """
    order: List[OrderedItem] = Field(description="List of items ordered and their details")

@tool(args_schema=OrderInfo)  
def calculate_total(order: list) -> str:  
    """
    Calculate the total price of the order.
    
    Parameters:  
    order (list): A list of dictionaries, where each dictionary represents an item in the order and  has 'price' and 'quantity' keys.  
  
    Returns:  
    str: A string representing the total price of the order. 
    """  
    sum = 0
    for item in order:
        sum += item['price']*item['quantity']
    return f"The total price of the order excluding tax is ${sum}"


In [13]:
from langchain_core.utils.function_calling import convert_to_openai_function

tool_list = [retrieve, calculate_total]
functions = [convert_to_openai_function(f) for f in tool_list]

In [14]:
import json

for f in functions:
    print(json.dumps(f, indent=4), "\n")

{
    "name": "retrieve",
    "description": "retrieve(question: str, k: int) -> str - Retrieve the top k documents from the Crypto vector store based on the user question",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "description": "Question inputted by the user to retrieve the top documents from the Crypto vector store or knowledge base",
                "type": "string"
            },
            "k": {
                "description": "The number of top documents to retrieve",
                "default": 10,
                "type": "integer"
            }
        },
        "required": [
            "question"
        ]
    }
} 

{
    "name": "calculate_total",
    "description": "calculate_total(order: list) -> str - Calculate the total price of the order.\n\nParameters:  \norder (list): A list of dictionaries, where each dictionary represents an item in the order and  has 'price' and 'quantity' keys.  \n\nReturns: 

LangChain has several abstractions to make working with agents easy.

### AgentAction
This is a dataclass that represents the action an agent should take. It has a tool property (which is the name of the tool that should be invoked) and a tool_input property (the input to that tool)

### AgentFinish
This represents the final result from an agent, when it is ready to return to the user. It contains a return_values key-value mapping, which contains the final agent output. Usually, this contains an output key containing a string that is the agent's response.

### Intermediate Steps
These represent previous agent actions and corresponding outputs from this CURRENT agent run. These are important to pass to future iteration so the agent knows what work it has already done. This is typed as a List[Tuple[AgentAction, Any]]. Note that observation is currently left as type Any to be maximally flexible. In practice, this is often a string.

### `AgentExecutor` is a built-in class that provides the `run_agent` functionality

```console
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = chain.invoke({
            "input": user_input, 
            "agent_scratchpad": format_to_openai_functions(intermediate_steps)
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "retrieve": retrieve, 
            "calculate_total": calculate_total
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

```

Also adds additional functionalities such as logging, error handling for tools and also the entire agent.

In [17]:
from langchain_openai.chat_models import AzureChatOpenAI
model = AzureChatOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment="gpt-4",
    streaming=True,
)

In [25]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain.agents.format_scratchpad import format_to_openai_functions

import random
from langchain_community.chat_message_histories.cosmos_db import CosmosDBChatMessageHistory

cosmos = CosmosDBChatMessageHistory(
    cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
    cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
    cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
    connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
    session_id="shiva-test" + str(random.randint(1, 10000)),
    user_id="shivac"
    )
# prepare the cosmosdb instance
cosmos.prepare_cosmos()

from langchain_core.callbacks.base import BaseCallbackHandler
class MyCallbackHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token, **kwargs) -> None:
        # # print every token on a new line
        # print(f"{token}")
        print(f"{token}", end="")

model_with_tools = model.bind(functions=functions)
    
prompt = ChatPromptTemplate.from_messages([
    (
        "system", 
        """
        You are a Coffee Ordering assistant for Contoso Coffee cafe and have access to two tools. Menu includes coffees, teas, bakery items, sandwiches and smoothies. As an agent, your job is to receive orders from customers and answer their inquiries about the menu items. 
        
        First tool is "retrieve" function. It can retrieve context based on user question from vector store that contains information about cafe's menu items. 
        You may need this tool to obtain information about the menu items and answer questions. Feel free to skip this tool if you don't need the context to answer the question. For example, when customer is greeting or when you can find answer from conversation history. 
        
        Second tool is "calculate_total" function. Since you are not an expert in math and calculations, use this tool to calculate the total price of the order (excluding tax) based on the items ordered and their prices.
        Do NOT use this tool until user specifically requests for the total price of the order. Skip this tool if you are not asked for the total price of the order.                       
        
        Please follow these instructions when interacting with customers to generate a good and brief conversation:
        • Since you are representing Contoso, NEVER MENTION 'I am an AI language model'. Always respond in first person, not in third person. Focus solely on cafe-related queries and ordering. 
        • Keep responses concise and clear. Long responses may disengage the customers. Do not provide additional information, such as description, until explicitly asked for. Do not repeat anything until asked for, especially if you already mentioned in the conversation history.
        • If a customer is asking for recommendations, provide only top 3 most relevant recommendations. Do not provide more than 3 as it may lead to a long response.
        • Inform the customer politely if an item they request is not available in the listed menu items below. If a customer likes to update or cancel the order, please help accordingly.
        • Capture any additional notes the customer may have for the menu items.
        • When listing an item in the order, mention the actual price of the item in parenthesis without multiplying it with the quantity.
        • At the end of each conversation, always obtain the customer's name, even if provided in a single word, and attach it to the order.
        • Once the customer provides their name, confirm it and state that the order can be picked up at our cafe in 15 minutes.
        • Contoso accepts only pickup orders. Payment is accepted only at the store, and if a customer suggests paying over the phone, inform them that payments are accepted solely at the store.
        Remember to always maintain a polite and professional tone.
        
        Conversaion History:
        {chat_history}
        """
                 
    ),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "User Input: {input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# from langchain.schema.agent import AgentFinish
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor

chain = prompt | model_with_tools | OpenAIFunctionsAgentOutputParser()

agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

# memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
memory = ConversationBufferMemory(k=10, return_messages=True,memory_key="chat_history",chat_memory=cosmos)

agent_executor = AgentExecutor(agent=agent_chain, tools=tool_list, verbose=False, memory=memory)

In [23]:
def remove_price(input_string: str) -> str:
    import re
    # Use regular expression to remove parentheses and their contents
    output_string = re.sub(r'\s*\([^)]*\)', '', input_string)
    return output_string

In [26]:
import time

print("Welcome to Contoso Coffee Inc! How can I assist you?\n")
while True:
    query = input()
    if query in ["quit", "exit"]:
        break
    start_time = time.time()
    result = agent_executor.invoke({"input": query})
    end_time = time.time()
    execution_time = end_time - start_time
    print("\nCustomer: {0}".format(query))
    print("Assistant: {0}".format(remove_price(result['output'])))
    print(f"\tTime taken to respond: {round(execution_time)} seconds")

Welcome to Contoso Coffee Inc! How can I assist you?


Customer: Hi, how can you help me?
Assistant: Hello! I'm here to assist you with your order at Contoso Coffee. You can ask me about our menu, place an order, or inquire about specific items. How can I assist you today?
	Time taken to respond: 6 seconds

Customer: what french options do you have?
Assistant: We have the following French options on our menu:

1. Pain au Chocolat - Classic French pastry filled with chocolate.
2. Croissant - Buttery, flaky pastry, freshly baked to golden perfection.
3. Chocolate Croissant - Flaky croissant filled with rich chocolate.

Would you like to order any of these?
	Time taken to respond: 23 seconds

Customer: can I order 2 of the first item and 1 of the third item?
Assistant: Absolutely, I have noted down your order:

1. 2 x Pain au Chocolat
2. 1 x Chocolate Croissant

Is there anything else you would like to add to your order?
	Time taken to respond: 9 seconds

Customer: Can I place an order fo

In [27]:
print(memory.buffer_as_str)

Human: Hi, how can you help me?
AI: Hello! I'm here to assist you with your order at Contoso Coffee. You can ask me about our menu, place an order, or inquire about specific items. How can I assist you today?
Human: what french options do you have?
AI: We have the following French options on our menu:

1. Pain au Chocolat - Classic French pastry filled with chocolate. ($3.5)
2. Croissant - Buttery, flaky pastry, freshly baked to golden perfection. ($2.0)
3. Chocolate Croissant - Flaky croissant filled with rich chocolate. ($3.0)

Would you like to order any of these?
Human: can I order 2 of the first item and 1 of the third item?
AI: Absolutely, I have noted down your order:

1. 2 x Pain au Chocolat ($3.5 each)
2. 1 x Chocolate Croissant ($3.0 each)

Is there anything else you would like to add to your order?
Human: Can I place an order for 1 Mocha, 1 lemon cake, 1 blueberry smoothie, and 1 Chicken sandwich?
AI: Sure, I've added the following items to your order:

1. 1 x Mocha ($4.5 ea

In [28]:
cosmos.messages

[HumanMessage(content='Hi, how can you help me?'),
 AIMessage(content="Hello! I'm here to assist you with your order at Contoso Coffee. You can ask me about our menu, place an order, or inquire about specific items. How can I assist you today?"),
 HumanMessage(content='what french options do you have?'),
 AIMessage(content='We have the following French options on our menu:\n\n1. Pain au Chocolat - Classic French pastry filled with chocolate. ($3.5)\n2. Croissant - Buttery, flaky pastry, freshly baked to golden perfection. ($2.0)\n3. Chocolate Croissant - Flaky croissant filled with rich chocolate. ($3.0)\n\nWould you like to order any of these?'),
 HumanMessage(content='can I order 2 of the first item and 1 of the third item?'),
 AIMessage(content='Absolutely, I have noted down your order:\n\n1. 2 x Pain au Chocolat ($3.5 each)\n2. 1 x Chocolate Croissant ($3.0 each)\n\nIs there anything else you would like to add to your order?'),
 HumanMessage(content='Can I place an order for 1 Moch